In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [7]:
batch_size = 64 # Batch size for training
epochs = 100 # Number of epochs to train
latent_dim = 256 # Latent Dimensionality of the encoding space
num_samples = 10000 # Number of samples to train 


In [3]:
import requests, zipfile, io
from pathlib import Path
data_url = 'http://www.manythings.org/anki/fra-eng.zip'

r = requests.get(data_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
Path("data/").mkdir(parents=True, exist_ok=True)
z.extractall("data/")

data_path = 'data/fra.txt'

In [27]:
# vectorize the data

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text, _ = line.split('\t')
  # print(input_text, target_text)
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end sequence" character

  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)

  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [28]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [29]:
print('Number of samples: {}'.format(len(input_texts)))
print('Number of unique input tokens: {}'.format(num_encoder_tokens))
print('Number of unique output tokens: {}'.format(num_decoder_tokens))
print('Max sequence length for inputs: {}'.format(max_encoder_seq_length))
print('Max sequence length for outputs: {}'.format(max_decoder_seq_length))

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [30]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [34]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [37]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1.

  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.

    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [42]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# we discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

In [46]:
## Set up the decoder, using 'encoder_states' as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [47]:
# define the model that will run
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 45s 337ms/step - loss: 2.2082 - accuracy: 0.9752 - val_loss: 2.4318 - val_accuracy: 0.9831
Epoch 2/100
125/125 [==============================] - 42s 335ms/step - loss: 2.2336 - accuracy: 0.9831 - val_loss: 2.6165 - val_accuracy: 0.9831
Epoch 3/100
125/125 [==============================] - 41s 329ms/step - loss: 2.3845 - accuracy: 0.9831 - val_loss: 2.7785 - val_accuracy: 0.9831
Epoch 4/100
125/125 [==============================] - 41s 331ms/step - loss: 2.5288 - accuracy: 0.9831 - val_loss: 2.9234 - val_accuracy: 0.9831
Epoch 5/100
125/125 [==============================] - 42s 333ms/step - loss: 2.6642 - accuracy: 0.9831 - val_loss: 3.0819 - val_accuracy: 0.9831
Epoch 6/100
125/125 [==============================] - 41s 332ms/step - loss: 2.8002 - accuracy: 0.9831 - val_loss: 3.2537 - val_accuracy: 0.9831
Epoch 7/100
125/125 [==============================] - 41s 331ms/step - loss: 2.9932 - accuracy: 0.9824 - val_loss: 3.4579 -